In [1]:
from library.Mag import *
from SimPEG import PF, Utils, Mesh
import matplotlib.pyplot as plt
import ipywidgets as widgets

# Preparation and initial assessment of magnetic data#


In this section, we will calculate the magnetic response of a simple geologic model and then grid the data, in preparation for application of a series of magnetic data filters. 

## The Geologic Model ###

For this demonstration, a simple 3D geologic model was built attempting to capture several different types of geologic bodies.

The geologic bodies are represented by three different blocks superimposed on a background:

**Block 1:** Large strongly magnetic domain, reflective of a magnetic plutonic complex or magnetic volcanic rock package

**Block 2:** Small moderately magnetic domain, reflective of a shallow magnetic intrusion

**Block 3:** Linear steeply-dipping non-magnetic feature, reflective of a fault zone along which magnetite-destruction has occurred

**Background:** Weakly magnetic background, reflective of weakly magnetic volcanic rocks  


The extents of the survey area are approximately 3 km x 3 km.


Figure, plan view, cross-section


-do the prism colors mean anything here? Color-wise, should we make it look geological or reflecting susceptibility?  
-view is looking west, can we rotate whole model (all prisms and survey 90 degrees clockwise)? Want to look north with east-west flight lines




In [3]:
# Create a survey
fileName = './DIGHEM_Mag_floor10nt_25m.obs'
xyzd = np.genfromtxt(fileName, skip_header=3)
xyzd[:,-2] = 0
B = np.r_[60308, 83.8, 25.4]
survey = Mag.createMagSurvey(xyzd, B)
cntr = np.mean(xyzd[:, :2], axis=0)

prisms = []

# Create a synthetic block model and place it at the center of the survey
prisms.append( Simulator.definePrism())
prisms[0].x0, prisms[0].y0, prisms[0].z0 = cntr[0]+100, cntr[1]-100, xyzd[:, 2].min() - 50
prisms[0].dx, prisms[0].dy, prisms[0].dz = 300, 300 ,300
prisms[0].pdec = -30
suscs = [0.01]

# The background
prisms.append( Simulator.definePrism())
prisms[1].x0, prisms[1].y0, prisms[1].z0 = cntr[0], cntr[1], xyzd[:, 2].min()-300
prisms[1].dx, prisms[1].dy, prisms[1].dz = 8000, 8000 ,2000
# prisms[1].pdec, prisms[1].pinc = -45, 20
suscs += [0.1]

# A long negative dyke
prisms.append( Simulator.definePrism())
prisms[2].x0, prisms[2].y0, prisms[2].z0 = cntr[0]-500, cntr[1]-200, xyzd[:, 2].min()-400
prisms[2].dx, prisms[2].dy, prisms[2].dz = 4000, 100 ,500
prisms[2].pdec, prisms[2].pinc = -75, 20
suscs += [-0.1]

# A large unit on the Eastern boundary
prisms.append( Simulator.definePrism())
prisms[3].x0, prisms[3].y0, prisms[3].z0 = cntr[0]+1500, cntr[1], xyzd[:, 2].min() - 200
prisms[3].dx, prisms[3].dy, prisms[3].dz = 5000, 1500 ,500
prisms[3].pdec, prisms[3].pinc = -60, 0
suscs += [0.05]


for prism, susc in zip(prisms, suscs):
    prob = Mag.problem(prism=prism, survey=survey)
    prob.susc = susc
    survey.dobs += prob.fields()[0]

def Prism(View_dip,View_azm,View_lim):

    Simulator.plotObj3D(prisms, survey, View_dip,View_azm,View_lim, colors=['w','','g','r'])
    
    
out = widgets.interactive(Prism,
                          View_dip=widgets.FloatSlider(min=0, max=90, step=1, value=90, continuous_update=False),
                          View_azm=widgets.FloatSlider(min=0, max=360, step=1, value=0, continuous_update=False),
                          View_lim=widgets.FloatSlider(min=1, max=2000, step=1, value=870, continuous_update=False),
                          )
# View prism
display(out)

interactive(children=(FloatSlider(value=90.0, continuous_update=False, description='View_dip', max=90.0, step=1.0), FloatSlider(value=0.0, continuous_update=False, description='View_azm', max=360.0, step=1.0), FloatSlider(value=870.0, continuous_update=False, description='View_lim', max=2000.0, min=1.0, step=1.0), Output()), _dom_classes=('widget-interact',))

## Magnetic response of a simple geologic model

The magnetic response (total field anomaly) of the described geologic model is calculated on a series of east-west flight lines running roughly perpendicular to the general strike of geologic boundaries and structures featured in the model. Flight line spacing is 200 m. Data was collected at a flight height of 40 m. Topography is assumed to be flat in this example. The magnetic data are in nanoteslas (NT). 

As discussed in Section 1, the magnetic response recorded will depend on the inclination, declination, and field strength of the magnetic field at the survey location. Use the drop down menu to explore a range of responses correlating to varied magnetic field parameters. The default values reflect inclination, declination, and field strength recorded in central British Columbia, Canada.  

A profile through the magnetic data can be viewed by clicking on the grey ‘Profile’ button, and in this example the profile location can be changed using the slider bars. 

-Not sure ‘width’ and ‘height’ are needed



In [4]:
param = Simulator.ViewMagSurvey2D(survey)
display(param)

interactive(children=(FloatSlider(value=558034.7817500001, continuous_update=False, description='East', max=560595.5952500001, min=555473.96825, step=10.0), FloatSlider(value=7133989.5, continuous_update=False, description='North', max=7136799.5, min=7131179.5, step=10.0), FloatSlider(value=2688.85417500005, continuous_update=False, description='Width', max=2688.85417500005, min=10.0, step=10.0), FloatSlider(value=2950.5, continuous_update=False, description='Height', max=2950.5, min=10.0, step=10.0), FloatSlider(value=0.0, continuous_update=False, description='Azimuth', max=90.0, min=-90.0, step=5.0), FloatSlider(value=1900.9120036039603, continuous_update=False, description='Length', max=1900.9120036039603, min=10.0, step=10.0), BoundedFloatText(value=20.0, description='Npts', min=10.0, step=1.0), ToggleButton(value=False, description='Profile'), Output()), _dom_classes=('widget-interact',))

## Gridding the magnetic data

Magnetic data must be gridded prior to applying transforms and filters. This means that the flight line data must be interpolated on a regular 2D grid. There are numerous possible ways to grid data. Several researchers have demonstrated that minimum curvature methods are robust and yield a sensible and smooth result, so we have chosen minimum curvature to complete the gridding for this example. In this example, the data are interpolated on a 10 m x 10 m grid.

The resulting grid shows the larger and smaller magnetic bodies, and the magnetic low related to the NE trending structure.

## Basic visualization

Simple enhancements to the image can help the user better visualize features within the magnetic data. Applying sun-shading from various angles, and experimenting with colour scales and colour stretches are a simple means to explore the data initially. 

Try using the slider bars below to change the direction (azdeg) and angle (altdeg) of sun shading, to change color saturation (alpha) and transparency (alphaHS), and to manipulate magnetic contours lines (contour)

-don’t think we need vScale





In [15]:
gridCC, dMinCurv = Simulator.minCurvatureInterp(
    xyzd[:,:2], survey.dobs,
    vectorX=None, vectorY=None, vectorZ=None, gridSize=10,
    tol=1e-5, iterMax=None, method='spline'
)

X = gridCC[:,0].reshape(dMinCurv.shape, order='F')
Y = gridCC[:,1].reshape(dMinCurv.shape, order='F')

In [18]:
def dataViewer(X,Y,dataGrid):
      
    def plotData(SunAzimuth,SunAngle,Contrast,Transparency,vScale,MagContour):
        fig = plt.figure(figsize=(8,6))
        axs= plt.subplot()

        # Read the data
#         vmin, vmax = -25, 150
        cmap = plt.cm.gist_earth

        # Add shading
        im, CS = Simulator.plotDataHillside(X, Y,dataGrid ,
                                  axs=axs, cmap='RdBu_r', 
                                  clabel=False, contour=MagContour,
                                  alpha=Contrast, alphaHS=Transparency, ve=vScale, azdeg=SunAzimuth, altdeg=SunAngle)

        # Add points at the survey locations
        plt.scatter(xyzd[:,0], xyzd[:,1], s=2, c='k')

        # Set limits and labels
#         axs.set_xlim([xmin,xmax])
#         axs.set_ylim([ymin,ymax])
        # axs.set_xticklabels([556750,558000, 559250], size=14)
        # axs.set_xticks([556750,558000, 559250])
        # axs.set_yticklabels([7133000,7134000, 7135000], size=14, rotation=90, va='center')
        plt.colorbar(im)
        # axs.set_yticks([7133000,7134000, 7135000])
        axs.set_xlabel("Easting (m)", size=14)
        axs.set_ylabel("Northing (m)", size=14)
        axs.grid('on', color='k', linestyle='--')
        plt.show()
        # Add colorbar
        # pos = axs.get_position() # Get the position of previous axes
        # axbar = plt.axes([pos.x0+.25, pos.y0+0.2,  pos.width*0.75, pos.height*.5]) # Create a new axes and reshape
        # cbar = plt.colorbar(im, orientation='vertical',format='%.3f') # Add a colorbar using the color definition of previous
        # cbar.set_ticks(np.linspace(-25,300,6)) # Change the tick position
        # cbar.set_ticklabels(np.round(np.linspace(-25,300,6))) # Change the tick labels
        # axbar.text(pos.x0+1., pos.y0+.75,'SI', size=14) # Add units and move it above
        # axbar.axis('off') # Only keep the colorbar, remove the rest

    out = widgets.interactive(plotData,
                              SunAzimuth=widgets.FloatSlider(min=0, max=360, step=5, value=0, continuous_update=False),
                              SunAngle=widgets.FloatSlider(min=0, max=90, step=5, value=45, continuous_update=False),
                              Contrast=widgets.FloatSlider(min=0, max=1, step=0.1, value=0.5, continuous_update=False),
                              Transparency=widgets.FloatSlider(min=0, max=1, step=0.1, value=1.0, continuous_update=False),
                              vScale=widgets.FloatSlider(min=1, max=1000, step=1., value=1.0, continuous_update=False),
                              MagContour=widgets.FloatSlider(min=0, max=20, step=1, value=10, continuous_update=False))
    return out

view = dataViewer(X, Y, dMinCurv)
display(view)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='SunAzimuth', max=360.0, step=5.0), FloatSlider(value=45.0, continuous_update=False, description='SunAngle', max=90.0, step=5.0), FloatSlider(value=0.5, continuous_update=False, description='Contrast', max=1.0), FloatSlider(value=1.0, continuous_update=False, description='Transparency', max=1.0), FloatSlider(value=1.0, continuous_update=False, description='vScale', max=1000.0, min=1.0, step=1.0), FloatSlider(value=10.0, continuous_update=False, description='MagContour', max=20.0, step=1.0), Output()), _dom_classes=('widget-interact',))